In [ ]:
# Configurações Iniciais e Imports
import os
import sys
import logging
import time
import trimesh
import numpy as np
import joblib

# Adicionar o diretório raiz do projeto ao path
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.core.mesh_processor import MeshProcessor
from src.core.detector_geometric import GeometricDetector
from src.core.detector_ml import MLDetector
from src.core.landmarks import LANDMARK_NAMES
from src.utils.visualization import plot_landmarks, plot_landmarks_2d
from src.utils.helpers import setup_logging, save_landmarks_to_json

# Configurar logging
setup_logging(log_level=logging.INFO)

print(f"Diretório de trabalho: {os.getcwd()}")
print(f"Path do módulo: {module_path}")
print(f"Landmarks definidos: {len(LANDMARK_NAMES)}")

In [ ]:
# Configurar diretórios
BASE_DIR = module_path
DATA_DIR = os.path.join(BASE_DIR, "data", "skulls")
CACHE_DIR = os.path.join(BASE_DIR, "data", "cache")
RESULTS_DIR = os.path.join(BASE_DIR, "results")
MODEL_DIR = os.path.join(BASE_DIR, "models")

# Criar diretórios se não existirem
for directory in [DATA_DIR, CACHE_DIR, RESULTS_DIR, MODEL_DIR]:
    os.makedirs(directory, exist_ok=True)

print(f"Diretórios configurados:")
print(f"  Dados: {DATA_DIR}")
print(f"  Cache: {CACHE_DIR}")
print(f"  Resultados: {RESULTS_DIR}")
print(f"  Modelos ML: {MODEL_DIR}")

In [ ]:
# Criar arquivo STL dummy para demonstração
dummy_stl_filename = "dummy_skull_demo.stl"
dummy_stl_path = os.path.join(DATA_DIR, dummy_stl_filename)

if not os.path.exists(dummy_stl_path):
    print(f"Criando arquivo STL dummy em {dummy_stl_path}")
    # Criar esfera com deformações para simular crânio
    mesh_dummy = trimesh.primitives.Sphere(radius=70, subdivisions=4)
    mesh_dummy.apply_scale([0.8, 1, 0.9])  # Deformar um pouco
    mesh_dummy.vertices += [10, 0, 70]  # Deslocar
    mesh_dummy.export(dummy_stl_path)
    print(f"✅ Arquivo STL dummy '{dummy_stl_filename}' criado")
else:
    print(f"✅ Usando arquivo STL existente: '{dummy_stl_filename}'")

TARGET_STL_FILENAME = dummy_stl_filename
print(f"Arquivo alvo: {TARGET_STL_FILENAME}")

In [ ]:
# Inicializar processador
processor = MeshProcessor(data_dir=DATA_DIR, cache_dir=CACHE_DIR)
TARGET_FACES = 2000  # Simplificar para melhor desempenho

print(f"Carregando e processando: {TARGET_STL_FILENAME}")

# Carregar malha original
start_time = time.time()
mesh_original = processor.load_skull(TARGET_STL_FILENAME, use_cache=True)
load_time = time.time() - start_time

if not mesh_original:
    print(f"❌ Falha ao carregar {TARGET_STL_FILENAME}")
    raise Exception("Não é possível continuar sem malha")

print(f"✅ Malha carregada em {load_time:.4f}s")
print(f"   Vértices originais: {len(mesh_original.vertices):,}")
print(f"   Faces originais: {len(mesh_original.faces):,}")

# Simplificar malha
start_time = time.time()
mesh_simplified = processor.simplify(
    mesh_original, 
    target_faces=TARGET_FACES, 
    use_cache=True, 
    original_filename=TARGET_STL_FILENAME
)
simplify_time = time.time() - start_time

if not mesh_simplified:
    print("⚠️  Falha na simplificação, usando malha original")
    mesh_simplified = mesh_original
else:
    print(f"✅ Malha simplificada em {simplify_time:.4f}s")
    print(f"   Vértices simplificados: {len(mesh_simplified.vertices):,}")
    print(f"   Faces simplificadas: {len(mesh_simplified.faces):,}")
    reduction = (1 - len(mesh_simplified.faces) / len(mesh_original.faces)) * 100
    print(f"   Redução: {reduction:.1f}%")

print(f"\n📊 Malha para detecção pronta: {len(mesh_simplified.vertices):,} vértices, {len(mesh_simplified.faces):,} faces")

In [ ]:
print("=== Executando Detecção Geométrica ===")

# Inicializar detector geométrico
detector_geom = GeometricDetector()

# Executar detecção
start_time = time.time()
landmarks_geometric = detector_geom.detect(mesh_simplified)
detection_time = time.time() - start_time

if landmarks_geometric:
    # Contar landmarks detectados
    detected_count = sum(1 for coords in landmarks_geometric.values() if coords is not None)
    total_count = len(landmarks_geometric)
    
    print(f"✅ Detecção geométrica concluída em {detection_time:.4f}s")
    print(f"📊 Landmarks detectados: {detected_count}/{total_count} ({detected_count/total_count*100:.1f}%)")
    
    print("\n--- Landmarks Detectados (Geométrico) ---")
    for name, coords in landmarks_geometric.items():
        if coords is not None:
            coord_str = f"[{coords[0]:.2f}, {coords[1]:.2f}, {coords[2]:.2f}]"
            print(f"  ✅ {name}: {coord_str}")
        else:
            print(f"  ❌ {name}: Não detectado")
    
    # Salvar resultados
    geom_result_path = os.path.join(
        RESULTS_DIR, 
        f"{os.path.splitext(TARGET_STL_FILENAME)[0]}_geometric_landmarks.json"
    )
    if save_landmarks_to_json(landmarks_geometric, geom_result_path):
        print(f"💾 Resultados geométricos salvos em: {geom_result_path}")
    
else:
    print("❌ Falha na detecção geométrica")
    landmarks_geometric = None

In [ ]:
print("=== Verificando Disponibilidade de Modelos ML ===")

# Inicializar detector ML
ml_detector = MLDetector(model_dir=MODEL_DIR)

# Verificar quais modelos estão disponíveis
available_models = []
for landmark_name in LANDMARK_NAMES:
    model_path = os.path.join(MODEL_DIR, f"rf_model_{landmark_name}.joblib")
    scaler_path = os.path.join(MODEL_DIR, f"scaler_{landmark_name}.joblib")
    if os.path.exists(model_path) and os.path.exists(scaler_path):
        available_models.append(landmark_name)

print(f"📊 Modelos ML disponíveis: {len(available_models)}/{len(LANDMARK_NAMES)}")

if available_models:
    print("✅ Modelos encontrados para:")
    for model in available_models:
        print(f"   - {model}")
else:
    print("⚠️  Nenhum modelo ML encontrado")
    print("🔧 Criando modelos dummy básicos para demonstração...")
    
    # Criar dados de treinamento dummy muito simples
    dummy_mesh_train = trimesh.primitives.Sphere(radius=50, center=[0, 0, 50])
    dummy_gt_train = {
        "Glabela": [0, 50, 50], 
        "Nasion": [0, 45, 40], 
        "Bregma": [0, 0, 100], 
        "Opisthocranion": [0, -50, 50], 
        "Euryon_Esquerdo": [-50, 0, 50], 
        "Euryon_Direito": [50, 0, 50], 
        "Vertex": [0, 0, 100], 
        "Inion": [0, -45, 40]
    }
    
    # Treinar modelos dummy apenas para alguns landmarks principais
    primary_landmarks = ["Glabela", "Bregma", "Euryon_Direito"]
    training_successful = True
    
    for name in primary_landmarks:
        if name in dummy_gt_train:
            print(f"   🔧 Treinando modelo dummy para: {name}")
            try:
                success = ml_detector.train([dummy_mesh_train], [dummy_gt_train], name)
                if success:
                    available_models.append(name)
                    print(f"      ✅ Modelo {name} criado")
                else:
                    print(f"      ❌ Falha ao criar modelo {name}")
                    training_successful = False
            except Exception as e:
                print(f"      ❌ Erro ao treinar {name}: {e}")
                training_successful = False
    
    if training_successful and available_models:
        print(f"✅ Modelos dummy criados com sucesso para {len(available_models)} landmarks")
    else:
        print("❌ Falha ao criar modelos dummy")

print(f"\n📊 Total de modelos disponíveis: {len(available_models)}")

In [ ]:
print("=== Executando Detecção por Machine Learning ===")

landmarks_ml = None

if available_models:
    # Executar detecção ML
    start_time = time.time()
    landmarks_ml = ml_detector.detect(mesh_simplified)
    ml_detection_time = time.time() - start_time
    
    if landmarks_ml:
        # Contar landmarks detectados
        ml_detected_count = sum(1 for coords in landmarks_ml.values() if coords is not None)
        
        print(f"✅ Detecção ML concluída em {ml_detection_time:.4f}s")
        print(f"📊 Landmarks detectados: {ml_detected_count}/{len(landmarks_ml)} ({ml_detected_count/len(landmarks_ml)*100:.1f}%)")
        
        print("\n--- Landmarks Detectados (Machine Learning) ---")
        for name, coords in landmarks_ml.items():
            if coords is not None:
                coord_str = f"[{coords[0]:.2f}, {coords[1]:.2f}, {coords[2]:.2f}]"
                print(f"  ✅ {name}: {coord_str}")
            else:
                print(f"  ❌ {name}: Não detectado / Modelo ausente")
        
        # Salvar resultados
        ml_result_path = os.path.join(
            RESULTS_DIR, 
            f"{os.path.splitext(TARGET_STL_FILENAME)[0]}_ml_landmarks.json"
        )
        if save_landmarks_to_json(landmarks_ml, ml_result_path):
            print(f"💾 Resultados ML salvos em: {ml_result_path}")
        
    else:
        print("❌ Falha na detecção ML")
        
else:
    print("❌ Não é possível executar detecção ML sem modelos treinados")
    print("💡 Para usar ML, treine modelos primeiro com dados reais")

In [ ]:
print("=== Gerando Visualizações Comparativas ===")

# Visualização do método geométrico
if landmarks_geometric:
    print("🎨 Gerando visualização geométrica...")
    vis_geom_path = os.path.join(
        RESULTS_DIR, 
        f"{os.path.splitext(TARGET_STL_FILENAME)[0]}_geometric_visualization_2d.png"
    )
    
    success_geom = plot_landmarks_2d(
        mesh_simplified, 
        landmarks_geometric, 
        title=f"Método Geométrico - {TARGET_STL_FILENAME}", 
        save_path=vis_geom_path
    )
    
    if success_geom:
        print(f"✅ Visualização geométrica salva em: {vis_geom_path}")
        
        # Exibir no notebook
        try:
            from IPython.display import Image, display
            print("\n=== Resultado Método Geométrico ===")
            display(Image(filename=vis_geom_path))
        except:
            print("Não foi possível exibir a imagem no notebook")
    else:
        print("❌ Falha na visualização geométrica")

# Visualização do método ML
if landmarks_ml:
    print("\n🎨 Gerando visualização ML...")
    vis_ml_path = os.path.join(
        RESULTS_DIR, 
        f"{os.path.splitext(TARGET_STL_FILENAME)[0]}_ml_visualization_2d.png"
    )
    
    success_ml = plot_landmarks_2d(
        mesh_simplified, 
        landmarks_ml, 
        title=f"Método Machine Learning - {TARGET_STL_FILENAME}", 
        save_path=vis_ml_path
    )
    
    if success_ml:
        print(f"✅ Visualização ML salva em: {vis_ml_path}")
        
        # Exibir no notebook
        try:
            from IPython.display import Image, display
            print("\n=== Resultado Método Machine Learning ===")
            display(Image(filename=vis_ml_path))
        except:
            print("Não foi possível exibir a imagem no notebook")
    else:
        print("❌ Falha na visualização ML")

# Caso nenhum método tenha funcionado
if not landmarks_geometric and not landmarks_ml:
    print("❌ Nenhum método de detecção produziu resultados válidos")
elif not landmarks_geometric:
    print("⚠️  Apenas o método ML produziu resultados")
elif not landmarks_ml:
    print("⚠️  Apenas o método geométrico produziu resultados")
else:
    print("✅ Ambos os métodos produziram resultados para comparação")

In [ ]:
print("=== Análise Comparativa dos Métodos ===")

if landmarks_geometric and landmarks_ml:
    print("\n📊 Comparação de Performance:")
    
    # Taxa de detecção
    geom_detected = sum(1 for coords in landmarks_geometric.values() if coords is not None)
    ml_detected = sum(1 for coords in landmarks_ml.values() if coords is not None)
    total_landmarks = len(LANDMARK_NAMES)
    
    print(f"Taxa de Detecção:")
    print(f"  Geométrico: {geom_detected}/{total_landmarks} ({geom_detected/total_landmarks*100:.1f}%)")
    print(f"  ML: {ml_detected}/{total_landmarks} ({ml_detected/total_landmarks*100:.1f}%)")
    
    # Landmarks detectados por ambos
    both_detected = []
    only_geom = []
    only_ml = []
    
    for name in LANDMARK_NAMES:
        geom_found = landmarks_geometric.get(name) is not None
        ml_found = landmarks_ml.get(name) is not None
        
        if geom_found and ml_found:
            both_detected.append(name)
        elif geom_found:
            only_geom.append(name)
        elif ml_found:
            only_ml.append(name)
    
    print(f"\n🎯 Concordância entre Métodos:")
    print(f"  Detectados por ambos: {len(both_detected)} ({both_detected})")
    print(f"  Apenas geométrico: {len(only_geom)} ({only_geom})")
    print(f"  Apenas ML: {len(only_ml)} ({only_ml})")
    
    # Distâncias entre detecções correspondentes
    if both_detected:
        print(f"\n📏 Distâncias entre Detecções Correspondentes:")
        for name in both_detected:
            geom_coords = np.array(landmarks_geometric[name])
            ml_coords = np.array(landmarks_ml[name])
            distance = np.linalg.norm(geom_coords - ml_coords)
            print(f"  {name}: {distance:.2f} mm")
    
    # Tempo de processamento
    if 'detection_time' in locals() and 'ml_detection_time' in locals():
        print(f"\n⏱️  Tempo de Processamento:")
        print(f"  Geométrico: {detection_time:.4f}s")
        print(f"  ML: {ml_detection_time:.4f}s")
        if detection_time > 0:
            speedup = ml_detection_time / detection_time
            if speedup > 1:
                print(f"  Geométrico é {speedup:.1f}x mais rápido")
            else:
                print(f"  ML é {1/speedup:.1f}x mais rápido")

elif landmarks_geometric:
    print("✅ Apenas método geométrico produziu resultados")
    print(f"   Taxa de detecção: {geom_detected}/{len(LANDMARK_NAMES)} ({geom_detected/len(LANDMARK_NAMES)*100:.1f}%)")
    
elif landmarks_ml:
    print("✅ Apenas método ML produziu resultados")
    print(f"   Taxa de detecção: {ml_detected}/{len(LANDMARK_NAMES)} ({ml_detected/len(LANDMARK_NAMES)*100:.1f}%)")
    
else:
    print("❌ Nenhum método produziu resultados válidos")

In [ ]:
print("=== CONCLUSÕES DA DEMONSTRAÇÃO ===")
print()
print("Este notebook demonstrou como utilizar os dois métodos de detecção implementados no sistema.")
print()
print("### Principais Observações:")
print()
print("#### Método Geométrico:")
print("- ✅ **Vantagens:** Rápido, não requer treinamento, baseado em princípios anatômicos")
print("- ⚠️ **Limitações:** Pode ser menos preciso, dependente de heurísticas")
print()
print("#### Método Machine Learning:")
print("- ✅ **Vantagens:** Potencialmente mais preciso com dados adequados, adaptável")
print("- ⚠️ **Limitações:** Requer modelos treinados, dependente da qualidade dos dados")
print()
print("### Recomendações:")
print()
print("1. **Para uso imediato:** Use o método geométrico que não requer treinamento")
print("2. **Para melhor precisão:** Treine modelos ML com dados reais representativos")
print("3. **Para robustez:** Combine ambos os métodos e use votação ou consenso")
print()